# Capstone Project - Battle of Neighborhoods
### Applied Data Science Capstone by IBM/Coursera
#### by Hans-Joachim Steinort

## Table of contents <a name="table_of_contents"></a>
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)
* [Literature](#literatur)

## Introduction: Business Problem <a name="introduction"></a> 
[Back to top](#table_of_contents)

As emerging start-up we are looking for an adequate city where we can open our headquarters. Of course we have the ambition to choose a city with international flair, a city renown all around the world, something like New York City or Tokyo. Unfortunately we are bound to the south of Germany. So no London, Paris or Singapur for us, not even Berlin. Luckily we are located near a city that proclaimed till 2005 to be a "world city with heart" - Munich [[1]](#l_1).

To verify this claim we will use the powers of data science to analyse the neighbourhoods of Munich and compare them to Manhattan, NYC as our baseline. If we find similarities we might be safe to choos Munich for our headquarters and still have the international flair we are looking for. 

## Data Aquisition<a name="data"></a>
[Back to top](#table_of_contents)

Based on our requirements the following factors will influence our decission:
* office rent and availability
* the distance to the next international airport / main train station
* the classification of the neighbourhood based on the venues in direct vicinity (so our employees do not have to leave early if the want to go out) 

Necessary data will be extracted/generated from/by:
* **Real estate reports** ([[2]](#l_2), [[3]](#l_3))
* **GeoPy**
* **Foursquare API**

The data acquisition part consists of
* [Office Rent Comparison](#rent_comparison)
* [Airport and Train Station Distance](#air_trian)
* [Munich Segmentation](#segmentation)
* [Munich Segment Data Aquisition](#segment_data)

### Office Rent Comparison<a name="rent_comparison"></a>
[Back to Data](#data)

At the first glance Munich is comparabily cheap to Manhattan. As claimed in [[2]](#l_2) the **availability** of offices in Manhattan was **9.8% in Q3 2019** with an acclaimed **rent** of **79.77 $/SF/YR**. To get comparable results we have to convert this value to the German standard of Euro(€) per squaremeter(SQM) per month(M).

To get SQM from SF we have to multiply the value by the factor of 10.764:

79.77 * 10.764 = 858,64 $/SQM/YR

To get the monthly rent we then have to divide the value by the factor of 12:

858,64 / 12 = 71.55 $/SQM/M

To get the monthly rent in Euro we mulitply with a generous estimate of the 2019 average Dollar/Euro exchange rate of 0.9 €/$:

71.55 * 0.9 = 64.40 €/SQM/M

If we take a look at the office market report of Munich in [[3]](#l_3) we might find our first discouragement. Even if we choose our office in the most expensive area of Munich - in the city center - we can only pay **shatterig 36% less** compared to the Manhattan average.

**Manhattan, NYC: 64.40 €/SQM/M**

**City Center, Munich: 41.00 €/SQM/M**

So no flexing with the height of our rent... But at least we have found the area, we want to compare to our neighbourhoods of Manhattan. 

### Airport and Train Station Distance<a name="air_train"></a>
[Back to Data](#data)

Being internationally active we need fast access to an international airport, because it is too time consuming sailing accross the Atlantic for each and every meeting. Yet trying our best to reduce our environmental footprint we are planning to travell inside Germany by train. For this purpose a nearby train station (within 10 mins by bike, e.g. 5 km) would be beneficial.

Both distances will be roughly measured by a straight line derived from the latitude & longitude values of our respective neighbourhood/areal centers

In [4]:
# imports for retrieving latitudes and longitudes
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my-application")

In [5]:
# imports for meassuring distances between latitudes and longitudes
from math import sin, cos, sqrt, atan2, radians

R = 6373.0 # approximate radius of earth [km]

#### Manhattan, NYC

As main international airport of NYC we assume **John F. Kennedy Airport (JFK)**. As main train station of Manhattan we assume the **New York Pennsylvania Station**.

In [6]:
# get location of the center of Manhattan

location_Manhattan = geolocator.geocode("Manhattan NYC")

print(location_Manhattan.address)
print("")

latitude_Manhattan = location_Manhattan.latitude
longitude_Manhattan = location_Manhattan.longitude


print((latitude_Manhattan, longitude_Manhattan))

Manhattan, New York County, New York, United States of America

(40.7896239, -73.9598939)


In [7]:
# get location of JFK

location_JFK = geolocator.geocode("JFK NYC")

print(location_JFK.address)
print("")

latitude_JFK = location_JFK.latitude
longitude_JFK = location_JFK.longitude

print((latitude_JFK, longitude_JFK))

John F. Kennedy International Airport, 167th Street, Rochdale Village, Queens, Queens County, New York, 11430, United States of America

(40.642947899999996, -73.7793733748521)


In [9]:
# get location of Penn Station

location_Penn = geolocator.geocode("Pennsylvania Station NYC")

print(location_Penn.address)
print("")

latitude_Penn = location_Penn.latitude
longitude_Penn = location_Penn.longitude

print((latitude_Penn, longitude_Penn))

Pennsylvania Station, 234, West 31st Street, Chelsea, Manhattan Community Board 5, Manhattan, New York County, New York, 10001, United States of America

(40.7502382, -73.9928111)


In [10]:
# get airport distance [km]

rLat_1 = radians(latitude_JFK)
rLat_2 = radians(latitude_Manhattan)
rLong_1 = radians(longitude_JFK)
rLong_2 = radians(longitude_Manhattan)

dLat = rLat_1 - rLat_2
dLong = rLong_1 - rLong_2

a = sin(dLat / 2)**2 + cos(rLat_1) * cos(rLat_2) * sin(dLong / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1-a))

distance = R * c

print("Distance: %.2f" % distance, "km")

Distance: 22.31 km


In [11]:
# get train station distance [km]

rLat_1 = radians(latitude_Penn)
rLat_2 = radians(latitude_Manhattan)
rLong_1 = radians(longitude_Penn)
rLong_2 = radians(longitude_Manhattan)

dLat = rLat_1 - rLat_2
dLong = rLong_1 - rLong_2

a = sin(dLat / 2)**2 + cos(rLat_1) * cos(rLat_2) * sin(dLong / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1-a))

distance = R * c

print("Distance: %.2f" % distance, "km")

Distance: 5.18 km


Our **benchmark** for an office in Manhattan is an approximate airport distance of **22.31 km** and a train station distance of **5.18 km**.

#### City Center, Munich, Germany

As main international airport of Munic we assume **Franz Joseph Strauss Airport (MUC)**. As main train station of Manhattan we assume the **Munich Main Station (MMS)**.

In [54]:
# get location of the center of Manhattan

location_Munich = geolocator.geocode("Center Munich Germany")

print(location_Munich.address)
print("")

latitude_Munich = location_Munich.latitude
longitude_Munich = location_Munich.longitude


print((latitude_Munich, longitude_Munich))

München, Bayern, Deutschland

(48.1371079, 11.5753822)


In [13]:
# get location of MUC

location_MUC = geolocator.geocode("MUC Airport Munich Germany")

print(location_MUC.address)
print("")

latitude_MUC = location_MUC.latitude
longitude_MUC = location_MUC.longitude

print((latitude_MUC, longitude_MUC))

Flughafen München, Lohstraße, Schwaigermoos, Oberding, Oberding (VGem), Landkreis Erding, Bayern, 85445, Deutschland

(48.35376735, 11.778011507058581)


In [14]:
# get location of MMS

location_MMS = geolocator.geocode("Munich Main Station Germany")

print(location_MMS.address)
print("")

latitude_MMS = location_MMS.latitude
longitude_MMS = location_MMS.longitude

print((latitude_MMS, longitude_MMS))

München Hauptbahnhof, Bezirksteil Ludwigsvorstadt-Kliniken, Stadtbezirk 02 Ludwigsvorstadt-Isarvorstadt, München, Bayern, 80335, Deutschland

(48.1407138, 11.556312490222421)


In [15]:
# get airport distance [km]

rLat_1 = radians(latitude_MUC)
rLat_2 = radians(latitude_Munich)
rLong_1 = radians(longitude_MMS)
rLong_2 = radians(longitude_MUC)

dLat = rLat_1 - rLat_2
dLong = rLong_1 - rLong_2

a = sin(dLat / 2)**2 + cos(rLat_1) * cos(rLat_2) * sin(dLong / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1-a))

distance = R * c

print("Distance: %.2f" % distance, "km")

Distance: 29.16 km


In [16]:
# get train station distance [km]

rLat_1 = radians(latitude_MMS)
rLat_2 = radians(latitude_Munich)
rLong_1 = radians(longitude_MMS)
rLong_2 = radians(longitude_Munich)

dLat = rLat_1 - rLat_2
dLong = rLong_1 - rLong_2

a = sin(dLat / 2)**2 + cos(rLat_1) * cos(rLat_2) * sin(dLong / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1-a))

distance = R * c

print("Distance: %.2f" % distance, "km")

Distance: 1.47 km


With an approximate distance of **29.16 km** the airport in Munich is slightly farther away from the desired office area but this is absolutly within our tollerance. Especially because the distance from the center to the main train station is only **1.47 km**.

### Munich Segmentation<a name="segmentation"></a>
[Back to Data](#data)

Due to the fact that we have no predefined segmentation of the city center of Munich (compared to the different neighbourhoods in Manhattan), we have to build our own segmentation.

For this purpose we create an **equally spaced grid within a ~4km radius of the center of Munich**. Each artificial neighbourhood will have a radius of 200 m.

The calculation will take place within a Cartesian 2D coordinate system. The conversion between the WGS84 spherical coordinate system and the UTM Cartesian coordinate system is inspired by the Capstone Example [[4]](#l_4).

In [17]:
import shapely
from pyproj import Proj, transform
import math

In [35]:
def projection():
    return Proj("+proj=utm +zone=32U, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

def lonlat_to_xy(lon, lat):
    proj_xy = projection()
    x, y = proj_xy(lon, lat)
    
    return x, y

def xy_to_lonlat(x, y):
    proj_lonlat = proj_xy = projection()
    lon, lat = proj_lonlat(x, y, inverse=True)
    
    return lon, lat

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    
    return math.sqrt(dx*dx + dy*dy)

In [57]:
munich_center_x, munich_center_y = lonlat_to_xy(longitude_Munich, latitude_Munich)

print('Munich city center UTMx={}, UTMy={}'.format(munich_center_x, munich_center_y))

Munich city center UTMx=691595.3564082251, UTMy=5334747.274427182


In the next step we build a **hexagonal grid** euqally spaced around the city center of Munich.

In [62]:
y_off = math.sqrt(3) / 2     # vertical offset
n_cnt_x = 21                   # neighbourhood horizontal counter 
n_off_y = int(n_cnt_x/y_off)     # neighbourhood vertical counter

circle_radius = 4000 #[m]
center_distance = 400 #[m]

x_min = munich_center_x - circle_radius
x_step = center_distance
y_min = munich_center_y - circle_radius - (n_off_y * y_off * center_distance - circle_radius * 2) / 2
y_step = center_distance * y_off

latitudes = []
longitudes = []
distances_from_center = []
X = []
Y = []

for i in range (0, n_off_y):
    y = y_min + i * y_step
    x_off = (center_distance / 2) if i%2 == 0 else 0
    for j in range (0, n_cnt_x):
        x = x_min + j * x_step + x_off
        distance_from_center = calc_xy_distance(munich_center_x, munich_center_y, x, y)
        if (distance_from_center <= circle_radius + 1):
            lon, lat = xy_to_lonlat(x, y)
            longitudes.append(lon)
            latitudes.append(lat)
            distances_from_center.append(distance_from_center)
            X.append(x)
            Y.append(y)
            
print(len(latitudes), 'neighbourhood centers generated.')

364 neighbourhood centers generated.


To verify our segmentation we generate a map with folium.

In [64]:
import folium

In [69]:
center_Munich = [location_Munich.latitude, location_Munich.longitude]
map_Munich = folium.Map(location=center_Munich, zoom_start=13)
folium.Marker(center_Munich, popup='City Center Munich').add_to(map_Munich)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=center_distance/2, color='green', fill=False).add_to(map_Munich)

map_Munich

For further analysis we add our newly created neighbourhoods to a pandas dataframe.

In [82]:
import pandas as pd

munich_data = pd.DataFrame({'Center Latitudes':latitudes,
                            'Center Longitudes':longitudes})

### Munich Segment Data Aquisition<a name="segment_data"></a>
[Back to Data](#data)

To create a baseline which we can compare to the clusters of the Manhattan neighbourhood clustering from the lab in week 3, we have to reproduce the venue aquisition with the **Foursquare API** in regard to our artificially created neighbourhood centers.

To achieve this we make use of the getNearbyVenues function from the lab. [[5]](#l_5).

In [91]:
import requests

In [97]:
# Credentials hidden

In [92]:
def getNearbyVenues(latitudes, longitudes, limit=100, radius=500):
    
    venues_list=[]
    for lat, lng in zip(latitudes, longitudes):            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
              CLIENT_ID, 
              CLIENT_SECRET, 
              VERSION, 
              lat, 
              lng, 
              radius, 
              limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([( 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Center Latitude', 
                             'Center Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [93]:
venue_limit = 100
search_radius = center_distance / 2

munich_venues = getNearbyVenues(latitudes=munich_data['Center Latitudes'],
                                longitudes=munich_data['Center Longitudes'],
                                limit=venue_limit,
                                radius=search_radius)

In [94]:
print(munich_venues.shape)
munich_venues.head()

(2561, 6)


,Center Latitude,Center Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,48.103219,11.557567,Isar Alm zum Gartl,48.102888,11.558148,Beer Garden
1,48.103099,11.562935,Reitclub Isartal,48.103324,11.562401,Stables
2,48.103099,11.562935,Dog's Academy Hundeschule,48.103610,11.562719,Dog Run
3,48.102980,11.568302,Therapie- und Trainingszentrum München Harlaching,48.103578,11.568077,Gym / Fitness Center
4,48.102980,11.568302,H Kurzstraße,48.104400,11.569698,Tram Station


This concludes our data aquisition phase. As last step we pickle our retrieved venue list so that we do not have to call the Foursquare API over and over again.

In [96]:
import pickle

munich_venues.to_pickle('./Munich_Venues.pkl')

## Methodology <a name="methodology"></a>
[Back to top](#table_of_contents)

**_To be added in Week 2_**

## Analysis <a name="analysis"></a>
[Back to top](#table_of_contents)

**_To be added in Week 2_**

## Results and Discussion <a name="results"></a>
[Back to top](#table_of_contents)

**_To be added in Week 2_**

## Conclusion <a name="conclusion"></a>
[Back to top](#table_of_contents)

**_To be added in Week 2_**

## Literature <a name="literatur"></a>
[Back to top](#table_of_contents)

[1] <a name="l_1"></a>https://www.muenchen.de/rathaus/Stadtverwaltung/Referat-fuer-Arbeit-und-Wirtschaft/Presse/Muenchen-mag-dich-Munich-loves-you.html

[2] <a name="l_2"></a>Colliers International - Manhattan Office Market Report 3Q 2019

[3] <a name="l_3"></a>JLL - Office Market Profile Munich 4th Quarter 2019

[4] <a name="l_4"></a>https://cocl.us/coursera_capstone_notebook

[5] <a name="l_5"></a>CognitiveClass.ai - Segmenting and Clustering Neighborhoods in New York City